# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
#city_weather_path = "output_data/city_weather.csv"
#vacations_weather = pd.read_csv(city_weather_path)
snow_weather = pd.read_csv("../Instructions/snow_weather.csv")
snow_weather

,Unnamed: 0,City,Temperature,Humidity,Cloudiness,Snow,wind_speed,Date,Country,Longitude,Latitude
0,0,nikolskoye,22.89,98,99,0.65,2.64,1643766852,RU,30.7861,59.7035
1,1,chapais,19.18,90,100,0.57,18.57,1643766861,CA,-74.8492,49.7834
2,2,petrozavodsk,-0.94,92,100,0.32,2.24,1643766862,RU,34.3469,61.7849
3,3,kortkeros,24.80,96,100,0.11,6.67,1643766866,RU,51.5806,61.8106
4,4,vardo,28.74,63,100,0.10,10.87,1643766871,NO,31.1107,70.3705
5,5,great falls,0.45,81,100,0.51,10.36,1643766780,US,-111.3008,47.5002
6,6,gari,21.38,95,100,0.16,8.79,1643766879,RU,62.3497,59.4307
7,7,yokote,32.16,87,94,0.27,13.96,1643766886,JP,140.5667,39.3000
8,8,lasa,34.25,94,100,0.12,9.28,1643766887,IT,10.7002,46.6166
9,9,sisimiut,6.75,89,100,0.24,11.36,1643766897,GL,-53.6735,66.9395


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
snow_weather.count()

Unnamed: 0     21
City           21
Temperature    21
Humidity       21
Cloudiness     21
Snow           21
wind_speed     21
Date           21
Country        21
Longitude      21
Latitude       21
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
#store latitude and longitude in locations
locations = snow_weather[["Latitude", "Longitude"]]
weight = snow_weather["Humidity"]

In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
max_humidity = snow_weather['Humidity'].max()
heat_layer = gmaps.heatmap_layer(locations, weights=
                                 humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig



NameError: name 'humidity' is not defined

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
snow_df = snow_weather.dropna()
snow_trimmed = snow_df.loc[snow_df['Temperature'] <= 32]
snow_trimmed_again =  snow_trimmed.loc[snow_trimmed['Temperature'] > 9] 
winter_vacations = snow_trimmed_again.loc[snow_trimmed['Snow'] > 0.3] 
#winter_vacations = vacations_trimmed_again.loc(vacations_trimmed['Cloudiness'] > 30  
winter_vacations

,Unnamed: 0,City,Temperature,Humidity,Cloudiness,Snow,wind_speed,Date,Country,Longitude,Latitude
0,0,nikolskoye,22.89,98,99,0.65,2.64,1643766852,RU,30.7861,59.7035
1,1,chapais,19.18,90,100,0.57,18.57,1643766861,CA,-74.8492,49.7834
11,11,pella,23.05,98,99,0.42,2.53,1643766911,RU,30.8181,59.7775
13,13,onega,11.46,94,98,0.35,5.46,1643766934,RU,38.1404,63.9061
19,19,esso,22.14,93,100,0.33,9.95,1643766951,RU,158.7000,55.9333


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = pd.DataFrame(winter_vacations, columns=["City", "Country", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,nikolskoye,RU,59.7035,30.7861,
1,chapais,CA,49.7834,-74.8492,
11,pella,RU,59.7775,30.8181,
13,onega,RU,63.9061,38.1404,
19,esso,RU,55.9333,158.7000,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))